In [6]:
# !pip install python-dotenv
from dotenv import load_dotenv
import os
# os.getcwd()

load_dotenv()
openai_api_key = os.environ.get("OPENAI_API_KEY")

In [10]:
# 3.0
# from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI #, ChatAnthropic

# llm = OpenAI()
chat = ChatOpenAI()

# a = llm.predict("How many planets are ther?")
# Error = The model `text-davinci-003` has been deprecated
b = chat.predict("How many planets are ther?")

b

'There are 8 planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.'

In [11]:
#3.1
chat = ChatOpenAI(
    temperature=0.1
)

from langchain.schema import HumanMessage, AIMessage, SystemMessage
messages = [
    SystemMessage(content="You are a geography expert. And you only reply in Italian."),
    AIMessage(content="Ciao, mi chiamo Paolo"),
    HumanMessage(content="Waht is the distance between Mexico and Thailand. Also what is you name?"),
]

chat.predict_messages(messages)
# AIMessage(content='La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri. Come posso aiutarti ancora?')

AIMessage(content='La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri. Come posso aiutarti ancora?')

In [19]:
# 3.2 prompt template
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(
    # temperature=0.1
)

template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "Ciao, mi chiamo {name}"),
    ("human", "Waht is the distance between {country_a} and {country_b}."),
])

prompt = template.format_messages(
    language = "Greek",
    name = "Socrates",
    country_a = "Mexico",
    country_b = "Thailand",
)
# template = PromptTemplate.from_template("Waht is the distance between {country_a} and {country_b}")
# prompt = template.format(country_a="Mexico", country_b="Thailand")

# chat.predict(prompt)
# 'The distance between Mexico and Thailand is approximately 15,000 kilometers (9,320 miles) when measured in a straight line.'
chat.predict_messages(prompt)


AIMessage(content='Η απόσταση μεταξύ Μεξικό και Ταϊλάνδης είναι περίπου 16.500 χιλιόμετρα.')

In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

In [23]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

p = CommaOutputParser()
p.parse("Hello,how, are,you")

['Hello', 'how', 'are', 'you']

In [27]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are list generating machine. Everything you are asked will be answered with a list of max {max_items}. Do not replay with anythin else"),
    ("human", "{question}")
])

# prompt = template.format_messages(
#     max_items = 10,
#     question = "What are the planets?"
# )

# result = chat.predict_messages(prompt)

# p = CommaOutputParser()
# p.parse(result.content)



['1. Mercury\n2. Venus\n3. Earth\n4. Mars\n5. Jupiter\n6. Saturn\n7. Uranus\n8. Neptune\n9. Pluto (dwarf planet)\n10. Eris (dwarf planet)']

In [28]:
chain = template | chat | CommaOutputParser()
# chain_one = template | chat | CommaOutputParser()
# chain_two = template | chat | CommaOutputParser()

# all = chain_one | chain_two | output

chain.invoke({
    "max_items": 10,
    "question": "What are the pokemons?"
})

['1. Pikachu\n2. Charizard\n3. Bulbasaur\n4. Squirtle\n5. Jigglypuff\n6. Eevee\n7. Snorlax\n8. Gengar\n9. Mewtwo\n10. Lucario']

In [30]:
# 3.4 Components
# Prompt - dict ~ promptValue, Retriever, LLM, ChatModel, Tool, OutputParser
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a chef. i create easy recipes with easy to find ingreditents"),
    ("human", "I want to cook {cuisine} food"),
])

chef_chain = chef_prompt | chat


In [31]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef. You find alternative ingredients with vegetables"),
    ("human", "{recipe}"),
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain
final_chain.invoke({
    "cuisine": "indian"
})


AIMessage(content="Here's a vegetarian twist on the classic Chicken Curry recipe using alternative ingredients with vegetables:\n\nIngredients:\n- 1 can of chickpeas, drained and rinsed\n- 1 onion, finely chopped\n- 2 cloves of garlic, minced\n- 1-inch piece of ginger, grated\n- 1 can of diced tomatoes\n- 1 can of coconut milk\n- 2 tablespoons of curry powder\n- 1 teaspoon of cumin\n- 1 teaspoon of turmeric\n- Salt and pepper to taste\n- Fresh cilantro for garnish\n- Cooked rice or naan bread for serving\n\nInstructions:\n1. In a large skillet, heat some oil over medium heat. Add the chopped onion and cook until softened, about 5 minutes.\n2. Add the minced garlic and grated ginger, and cook for another minute until fragrant.\n3. Add the chickpeas to the skillet and cook for a few minutes to allow them to absorb the flavors.\n4. Stir in the curry powder, cumin, turmeric, salt, and pepper, and cook for another minute.\n5. Pour in the diced tomatoes and coconut milk, and bring the mixtur

In [ ]:
# 3.5 Recap